In [ ]:
#load Models and inspects Params

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# === Utility Function to Count Parameters ===
def count_params(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

# === Load base ResNet and get feature size ===
base_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_features = base_model.fc.in_features

# --- 1) Pretrained baseline: "pure reuse" (we don't train it in our project) ---
model_pretrained = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
# Keep original 1000-class head or (if we prefer) swap to 10-class; either way:
for p in model_pretrained.parameters():
    p.requires_grad = False    # conceptually: 0 trainable params in our project

# --- 2) Feature-extracted model (NB02): backbone frozen, head trainable ---
model_feature = models.resnet18(weights=None)
model_feature.fc = nn.Linear(num_features, 10)
model_feature.load_state_dict(
    torch.load("./checkpoints/resnet18_feature_extraction.pth", weights_only=False, map_location="cpu")
)

# Freeze everything first
for p in model_feature.parameters():
    p.requires_grad = False
# Unfreeze only classifier head (this is what we *trained* in NB02)
for p in model_feature.fc.parameters():
    p.requires_grad = True

# --- 3) Fine-tuned model (NB03): layer4 + head trainable ---
model_finetuned = models.resnet18(weights=None)
model_finetuned.fc = nn.Linear(num_features, 10)
model_finetuned.load_state_dict(
    torch.load("./checkpoints/resnet18_finetuned.pth", weights_only=False, map_location="cpu")
)

# Freeze all layers first
for p in model_finetuned.parameters():
    p.requires_grad = False
# Unfreeze only layer4 and head (this is what we *trained* in NB03)
for p in model_finetuned.layer4.parameters():
    p.requires_grad = True
for p in model_finetuned.fc.parameters():
    p.requires_grad = True

# === Inspect which parameters are trainable ===
def print_trainable_layers(model, name):
    trainable_layers = [n for n, p in model.named_parameters() if p.requires_grad]
    print(f"\nModel: {name}")
    print(f"Trainable layers: {len(trainable_layers)} out of {len(list(model.parameters()))}")
    print("Example trainable params:",
          trainable_layers[-5:] if trainable_layers else "None")

print_trainable_layers(model_pretrained, "Pretrained (ImageNet)")
print_trainable_layers(model_feature, "Feature-Extracted (NB02)")
print_trainable_layers(model_finetuned, "Fine-Tuned (NB03)")

# === Count parameters ===
for name, mdl in zip(
    ["Pretrained", "Feature-Extracted", "Fine-Tuned"],
    [model_pretrained, model_feature, model_finetuned]
):
    total, trainable = count_params(mdl)
    print(f"{name:18s} | Total: {total/1e6:.2f}M | "
          f"Trainable: {trainable/1e6:.4f}M | "
          f"Frozen: {(total-trainable)/1e6:.4f}M")


In [ ]:
load test split 

In [ ]:
import torch
from torchvision import datasets, transforms
from pathlib import Path

# Paths (must match earlier notebooks)
DATA_ROOT = Path("./data/caltech101_10")
MAP_PATH  = Path("./checkpoints/class_to_idx.pth")

assert DATA_ROOT.exists(), f"Dataset not found at {DATA_ROOT}"
assert (DATA_ROOT / "test").exists(), f"Test split missing at {DATA_ROOT/'test'}"
assert MAP_PATH.exists(), f"class_to_idx mapping not found at {MAP_PATH}"

# Load saved mapping from NB01
class_to_idx_saved = torch.load(MAP_PATH, weights_only=False)
idx_to_class_saved = {v: k for k, v in class_to_idx_saved.items()}
class_names = [idx_to_class_saved[i] for i in range(len(idx_to_class_saved))]
num_classes = len(class_names)
print(f"[info] Saved classes ({num_classes}): {class_names}")

# Preprocessing (must match training notebooks)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
eval_tfms = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Build test dataset/loader
test_ds = datasets.ImageFolder(DATA_ROOT / "test", transform=eval_tfms)
from torch.utils.data import DataLoader
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)

print(f"[info] Test set size: {len(test_ds)} images")
print(f"[info] Test class_to_idx (from disk): {test_ds.class_to_idx}")

# Verify mapping consistency (order and names)
assert set(test_ds.class_to_idx.keys()) == set(class_to_idx_saved.keys()), \
    "Mismatch between saved classes and test folder classes."

# Build ordered list according to saved mapping (0..K-1)
ordered_from_saved = [name for name, _ in sorted(class_to_idx_saved.items(), key=lambda kv: kv[1])]
ordered_from_disk  = [name for name, _ in sorted(test_ds.class_to_idx.items(), key=lambda kv: kv[1])]
print(f"[check] Order (saved): {ordered_from_saved}")
print(f"[check] Order (disk) : {ordered_from_disk}")

assert ordered_from_saved == ordered_from_disk, \
    "Index order differs between saved mapping and current dataset. Please align folder names or mapping."

print("[ok] Class index order is consistent with saved mapping.")


In [ ]:
# Quick sanity peak at test samples

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Unnormalize helper for display
def unnormalize(img, mean=IMAGENET_MEAN, std=IMAGENET_STD):
    img = img.clone()
    for c, (m, s) in enumerate(zip(mean, std)):
        img[c] = img[c] * s + m
    return img.clamp(0, 1)

xb, yb = next(iter(test_loader))
k = min(8, xb.size(0))

plt.figure(figsize=(12, 3))
for i in range(k):
    img_disp = unnormalize(xb[i]).permute(1, 2, 0).numpy()
    plt.subplot(1, k, i + 1)
    plt.imshow(img_disp)
    plt.title(class_names[yb[i].item()], fontsize=9)
    plt.axis("off")
plt.suptitle("Sample images from the test split", fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# CT_Task 2 – Show 4 sample "airplanes" images from the test set

# 1. Class names from test dataset
class_names = test_ds.classes

# 2. Class index for "airplanes"
CT_airplane_class_name = "airplanes"
CT_airplane_class_idx = class_names.index(CT_airplane_class_name)

# 3. Collect indices of airplane images
CT_airplane_indices = [i for i, (_, label) in enumerate(test_ds) if label == CT_airplane_class_idx]

# Pick first 4 samples
CT_airplane_indices = CT_airplane_indices[:4]

# 4. Plotting
CT_airplane_fig, CT_airplane_axes = plt.subplots(1, 4, figsize=(12, 3))

for ax, idx in zip(CT_airplane_axes, CT_airplane_indices):
    img, label = test_ds[idx]
    
    # Convert CHW → HWC
    img_np = img.numpy().transpose(1, 2, 0)

    # Denormalization (ImageNet stats)
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    img_np = std * img_np + mean
    img_np = np.clip(img_np, 0, 1)

    ax.imshow(img_np)
    ax.set_title(class_names[label])
    ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# load the three models robustly

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
CKPT = Path("./checkpoints")
CKPT_FE = CKPT / "resnet18_feature_extraction.pth"
CKPT_FT = CKPT / "resnet18_finetuned.pth"
CKPT_PRE = CKPT / "resnet18_pretrained.pth"          # optional (if you created this)
CKPT_INIT = CKPT / "resnet18_frozen_init.pth"        # saved in NB01

num_classes = len(class_names)

def make_resnet18(num_classes: int, weights="IMAGENET1K_V1"):
    m = models.resnet18(weights=getattr(models.ResNet18_Weights, weights) if isinstance(weights, str) else weights)
    in_f = m.fc.in_features
    m.fc = nn.Linear(in_f, num_classes)
    return m

# --- Pretrained baseline: prefer user-saved ckpt; else NB01 init; else random head on ImageNet backbone
if CKPT_PRE.exists():
    model_pretrained = make_resnet18(num_classes, "IMAGENET1K_V1")
    model_pretrained.load_state_dict(torch.load(CKPT_PRE, weights_only=False, map_location="cpu"), strict=True)
    baseline_name = "pretrained.pth"
elif CKPT_INIT.exists():
    model_pretrained = make_resnet18(num_classes, "IMAGENET1K_V1")
    model_pretrained.load_state_dict(torch.load(CKPT_INIT, weights_only=False, map_location="cpu"), strict=True)
    baseline_name = "frozen_init.pth"
else:
    model_pretrained = make_resnet18(num_classes, "IMAGENET1K_V1")  # random 10-class head
    baseline_name = "ImageNet backbone + random 10-cls head"

model_pretrained = model_pretrained.to(device).eval()

# --- Feature extraction model (NB02)
assert CKPT_FE.exists(), f"Missing NB02 checkpoint: {CKPT_FE}"
model_feature = make_resnet18(num_classes, "IMAGENET1K_V1")
model_feature.load_state_dict(torch.load(CKPT_FE, weights_only=False, map_location="cpu"), strict=True)
# emulate FE inference setup (freezing not required for eval, but explicit)
for p in model_feature.parameters(): p.requires_grad = False
model_feature = model_feature.to(device).eval()

# --- Fine-tuned model (NB03)
assert CKPT_FT.exists(), f"Missing NB03 checkpoint: {CKPT_FT}"
model_finetuned = make_resnet18(num_classes, "IMAGENET1K_V1")
model_finetuned.load_state_dict(torch.load(CKPT_FT, weights_only=False, map_location="cpu"), strict=True)
model_finetuned = model_finetuned.to(device).eval()

print("[ok] Loaded models:")
print(f"  - Baseline: {baseline_name}")
print(f"  - Feature-extracted: {CKPT_FE.name}")
print(f"  - Fine-tuned:        {CKPT_FT.name}")


In [ ]:
# inference helper and batch prediction

In [ ]:
import torch.nn.functional as F
import numpy as np

@torch.no_grad()
def predict_batch(model, xb):
    logits = model(xb.to(device))
    probs = F.softmax(logits, dim=1).cpu().numpy()
    pred_idx = probs.argmax(axis=1)
    conf = probs.max(axis=1)
    return pred_idx, conf

# get one batch
xb, yb = next(iter(test_loader))
xb_dev = xb.to(device)

pred_pre, conf_pre = predict_batch(model_pretrained, xb)
pred_fe,  conf_fe  = predict_batch(model_feature,   xb)
pred_ft,  conf_ft  = predict_batch(model_finetuned, xb)

y_true = yb.numpy()

In [ ]:
# side by side visulization

In [ ]:
import matplotlib.pyplot as plt

def unnormalize(img_tensor, mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]):
    t = img_tensor.clone()
    for c,(m,s) in enumerate(zip(mean,std)):
        t[c] = t[c]*s + m
    return t.clamp(0,1)

k = min(12, xb.size(0))
cols = 3  # show 3 columns per row (we'll stack rows)
rows = k

plt.figure(figsize=(10, rows * 2.0))
for i in range(k):
    img = unnormalize(xb[i]).permute(1,2,0).numpy()
    t_lbl  = class_names[y_true[i]]
    p_pre  = class_names[pred_pre[i]]
    p_fe   = class_names[pred_fe[i]]
    p_ft   = class_names[pred_ft[i]]

    # one row per image with 1 big panel
    ax = plt.subplot(rows, 1, i+1)
    ax.imshow(img)
    title = (
        f"T: {t_lbl} | "
        f"PRE: {p_pre} ({conf_pre[i]:.2f}) | "
        f"FE: {p_fe} ({conf_fe[i]:.2f}) | "
        f"FT: {p_ft} ({conf_ft[i]:.2f})"
    )
    ax.set_title(title, fontsize=9)
    ax.axis("off")

plt.suptitle("Batch predictions — True vs Pretrained (PRE), Feature-Extracted (FE), Fine-Tuned (FT)", fontsize=12)
plt.tight_layout(rect=[0,0,1,0.98])
plt.show()


In [ ]:
# tabular comparsion

In [ ]:
import pandas as pd

rows = []
for i in range(k):
    rows.append({
        "true": class_names[y_true[i]],
        "PRE_pred": class_names[pred_pre[i]],
        "PRE_conf": f"{conf_pre[i]:.2f}",
        "FE_pred":  class_names[pred_fe[i]],
        "FE_conf":  f"{conf_fe[i]:.2f}",
        "FT_pred":  class_names[pred_ft[i]],
        "FT_conf":  f"{conf_ft[i]:.2f}",
    })

pd.DataFrame(rows)

In [ ]:
# compute metrics of all three models

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np

def evaluate_model(model, loader, name):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(yb.cpu().numpy())
    all_preds, all_labels = np.array(all_preds), np.array(all_labels)

    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average=None, zero_division=0
    )
    prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="macro", zero_division=0
    )
    prec_weight, rec_weight, f1_weight, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="weighted", zero_division=0
    )
    cm = confusion_matrix(all_labels, all_preds)
    return {
        "name": name,
        "acc": acc,
        "prec_macro": prec_macro,
        "rec_macro": rec_macro,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weight,
        "cm": cm
    }

results = []
for name, model in [
    ("Pretrained", model_pretrained),
    ("Feature-Extracted", model_feature),
    ("Fine-Tuned", model_finetuned)
]:
    print(f"Evaluating {name}...")
    res = evaluate_model(model, test_loader, name)
    results.append(res)
    print(f"{name} done.")


In [ ]:
# compare metrics of all three tables

In [ ]:
import pandas as pd

df_metrics = pd.DataFrame([{
    "Model": r["name"],
    "Accuracy": f"{r['acc']*100:.2f}%",
    "F1 (Macro)": f"{r['f1_macro']*100:.2f}%",
    "F1 (Weighted)": f"{r['f1_weighted']*100:.2f}%"
} for r in results])

display(df_metrics)

In [ ]:
# visualize comparsion side by side